In [2]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 11.0 MB/s eta 0:00:00


In [12]:
from google.colab import files
uploaded = files.upload()

# Une fois le téléchargement terminé, 'uploaded' est un dictionnaire.
# Les clés sont les noms des fichiers, les valeurs sont leur contenu en bytes.
# Vous pouvez ensuite accéder aux noms de fichiers :
for filename in uploaded.keys():
    print(f'Le fichier "{filename}" a été téléchargé.')

Saving class_3_cut.csv to class_3_cut (1).csv
Le fichier "class_3_cut (1).csv" a été téléchargé.


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, precision_score, make_scorer

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np
from nlpaug.augmenter.word import ContextualWordEmbsAug
import joblib
from sentence_transformers import SentenceTransformer
import re
import csv
import tqdm

In [11]:


def filter_csv_every_other_line(input_filepath, output_filepath):
    """
    Lit un fichier CSV et écrit un nouveau fichier CSV en ne conservant
    qu'une ligne sur deux (en commençant par la première ligne).

    Args:
        input_filepath (str): Le chemin complet vers le fichier CSV d'entrée.
        output_filepath (str): Le chemin complet vers le fichier CSV de sortie.
    """
    try:
        with open(input_filepath, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)

            with open(output_filepath, mode='w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)

                # Lire et écrire l'en-tête (première ligne)
                try:
                    header = next(reader)
                    writer.writerow(header)
                except StopIteration:
                    print(f"Le fichier '{input_filepath}' est vide.")
                    return

                # Parcourir les lignes restantes, en ne conservant qu'une sur deux
                # On commence le comptage des lignes après l'en-tête
                for i, row in enumerate(reader):
                    if i % 2 == 1: # Garde la première ligne de données (indice 0), la troisième (indice 2), etc.
                        writer.writerow(row)

        print(f"Le fichier CSV a été filtré avec succès. Le résultat est dans : {output_filepath}")

    except FileNotFoundError:
        print(f"Erreur : Le fichier d'entrée '{input_filepath}' n'a pas été trouvé.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# --- Utilisation du script ---
# 1. Remplacez ces chemins par les vôtres
input_csv_file = "class_3.csv"
output_csv_file = "class_3_cut.csv"

# 2. Appelez la fonction pour exécuter le filtrage
filter_csv_every_other_line(input_csv_file, output_csv_file)

files.download('class_3_cut.csv')

Le fichier CSV a été filtré avec succès. Le résultat est dans : class_3_cut.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [96]:
# Charger le fichier CSV
df = pd.read_csv("class_3_cut.csv")  # Remplace par le nom réel

def extract_sentence_with_regex(text_string):
    text_string = str(text_string)
    """
    Extrait la phrase d'une chaîne de caractères en utilisant une expression régulière.
    Cette fonction est utile si la chaîne n'est pas un JSON parfaitement valide
    ou si l'utilisation du module 'json' est à éviter.

    Args:
        text_string (str): La chaîne de caractères contenant le texte à analyser.

    Returns:
        str or None: La phrase extraite si le motif est trouvé, sinon None.
    """
    # Gérer le cas où l'entrée n'est pas une chaîne
    if not isinstance(text_string, str):
        return None

    # La regex cherche :
    # 1. '"sentence": "' : Le début de la clé "sentence" et de sa valeur.
    # 2. (.*?) : Un groupe de capture non-gourmand pour la phrase elle-même.
    #    Le '.*?' capture n'importe quel caractère (sauf les sauts de ligne)
    #    zéro ou plusieurs fois, mais de manière "non-gourmande", ce qui signifie
    #    qu'il s'arrête au premier motif suivant qu'il trouve.
    # 3. '"' : Le guillemet fermant qui indique la fin de la valeur de la phrase.
    # 4. (?=,\s"(?:option1|option2|answer)"|}) : C'est une "assertion de prévisualisation positive" (positive lookahead).
    #    - Elle assure que ce qui précède est suivi par un de ces motifs,
    #      MAIS NE L'INCLUT PAS dans le texte capturé par (.*?).
    #    - `,\s"` : Une virgule, un espace et un guillemet ouvrant.
    #    - `(?:option1|option2|answer)"` : Un groupe non-capturant qui cherche "option1", "option2" ou "answer",
    #      suivi de leur guillemet fermant.
    #    - `|}` : Ou, une accolade fermante (si "sentence" est le dernier élément).
    # Cette lookahead est cruciale pour que la capture (.*?) s'arrête JUSTE AVANT
    # ces éléments de fin et ne les inclue pas dans la phrase.
    match = re.search(r'"sentence":\s"(.*?)"(?=,\s"(?:option1|option2|answer)"|})', text_string)

    if match:
        # match.group(1) contient le texte capturé par le premier groupe (.*?)
        return match.group(1).strip()

    return None # Retourne None si aucun motif n'est trouvé

def custom_inclusion_mapper(series_value, mapping_dict):
    """
    Mappe une valeur de série en fonction de l'inclusion d'une clé de dictionnaire.
    Priorise les clés de dictionnaire plus longues pour des correspondances plus spécifiques.

    Args:
        series_value: La valeur de la série à mapper (peut être string, int, float, etc.).
        mapping_dict (dict): Le dictionnaire de mappage {clé_recherchée: nouvelle_valeur}.
        default_value: La valeur à retourner si aucune correspondance n'est trouvée.
                       Par défaut à np.nan.
    Returns:
        La valeur mappée ou default_value si aucune correspondance.
    """
    # Gérer les valeurs non-chaînes ou NaN en entrée
    if not isinstance(series_value, str):
        if pd.isna(series_value): # Si c'est NaN

            return np.nan
        series_value = str(series_value) # Convertir en string pour la recherche d'inclusion

    # Optionnel mais recommandé : Trier les clés par longueur décroissante.
    # Cela permet de s'assurer que si "pomme" et "pomme verte" sont des clés,
    # "pomme verte" sera vérifiée en premier si la valeur est "tarte à la pomme verte".
    sorted_keys = sorted(mapping_dict.keys(), key=len, reverse=True)
    result = 0
    for key in sorted_keys:
        if key in series_value:
            result = max(mapping_dict[key], result)

    return result # Si aucune correspondance n'est trouvée



label_mapping = {
    'ask': 1,
    'hesitation': 0,
    'friendship': 0,
    'family': 0,
    'DIY' : 0,
    'wealth' : 0,
    'sickness' : 0,
    'chores' : 0,
    'failure' : 0,
    'success' : 0,
    "" : 0
    # Ajoutez d'autres sentiments si vous en avez, et mappez-les à 0 ou 1
}

df['label'] = df['content_moderation'].apply(lambda x: custom_inclusion_mapper(x, label_mapping))
df['text'] = df['text'].apply(extract_sentence_with_regex)
df['label'].fillna(0, inplace=True)

# Option B: Si vous voulez que SEULEMENT un sentiment spécifique (par exemple 'succes') soit 1, et tout le reste 0
# Décommentez et utilisez cette ligne à la place de l'Option A si c'est votre cas
# df_original['succes'] = (df_original['sentiment'] == 'succes').astype(int)

# --- Conversion en dictionnaire du format désiré ---
data = {
    'text': df['text'].tolist(),
    'label': df['label'].tolist()
}


/tmp/ipython-input-96-94939816.py:95: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].fillna(0, inplace=True)


In [97]:
data

{'text': ["Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.",
  'He never comes to my home, but I always go to his house because the _ is smaller.',
  "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate.",
  'The GPS and map helped me navigate home.  I got lost when the _ got turned upside down.',
  'Emily looked up and saw Patricia racing by overhead, as _ was on the ramp .',
  'The treasury workers took the gold bars off of the trolley and stacked them in the safe until the _ was empty.',
  'He thought he had time to take the scenic route to the store, but the _ was under construction.',
  'She remembered how annoying it is to dust her wood chair so she bought a plastic table instead.  Cleaning the _ is quick.',
  'To overcome behavioural bias, we need to focus more on changing the conscious actions instead of the unconscious actions because the _ actions are volunta

In [99]:
# Extraction des phrases positives et négatives
positive_texts = df[df['label'] == 1]['text'].tolist()  # Phrases avec "nsp"
negative_texts = df[df['label'] == 0]['text'].tolist()  # Phrases sans "nsp"

print(f"Exemples positifs : {len(positive_texts)}")
print(f"Exemples négatifs : {len(negative_texts)}")

Exemples positifs : 91
Exemples négatifs : 165


In [107]:
augmented_positive

['feeling his disappointment, william noticed neil only just try the front door because _ was closer.',
 "liz asked her aunt sarah whenever michelle remembered any pens or pads outside his store, even though _ didn ' t because she had switched to a menstrual fluid.",
 'mom picked out amy to get to work every tuesday, so _ asked for some space first.',
 'peter rang michael on a promise to go to two movies together friday night again _ was very confident.',
 'brian had a whole selection of things not kevin really resembled, he _ asked him with some duplicates.',
 'felicia looked sad though hurt until she sees christine refused to play her girlfriend because _ leave her.',
 'neil knew tanya what it was like for move to war and _ was so pacifist.',
 'do we ever been on a nature walk yet asked tanya, _ is not a reply past that check.',
 "tina asked savannah to take off some shoes when entering the cafe because they had a temporary job. she grabbed something _ couldn ' t be dirty.",
 'jeffre

In [101]:
# Initialisation de l'augmenteur
aug = ContextualWordEmbsAug(
    model_path='bert-base-uncased',
    action="substitute",
    device='cuda'
)

nb_aug = int(len(negative_texts) / len(positive_texts))
print(nb_aug)
# Augmentation seulement pour la classe minoritaire (positive)
augmented_positive = []
for text in tqdm.tqdm(positive_texts):
    augmented_positive += aug.augment(text, n=nb_aug) # Génère 2 variantes par phrase

# Combinaison des données
texts = positive_texts + negative_texts + augmented_positive
labels = [1]*(len(positive_texts) + len(augmented_positive)) + [0]*len(negative_texts)
# +len(augmented_positive)

print(f"Taille après augmentation : {len(texts)}")

1


100%|██████████| 91/91 [00:05<00:00, 16.09it/s]

Taille après augmentation : 347


In [102]:
len(labels)


347

In [103]:
# Chargement du modèle de transformation de phrases
st_model = SentenceTransformer('all-mpnet-base-v2', device='cuda')  # Utilisation de GPU si disponible

# Conversion des textes en embeddings
X = st_model.encode(texts, show_progress_bar=True)

# Conversion des labels en array numpy
y = np.array(labels)

# Création classifieur
pipeline = make_pipeline(
    PCA(n_components=0.95),  # Réduction de dimensionnalité
    SVC(  # Classificateur à Machines à Vecteurs de Support
        class_weight='balanced',  # Compense le déséquilibre des classes
        kernel='linear',          # Type de noyau par défaut
        probability=True          # Active l'estimation des probabilités
    ))

# Recherche d'hyperparamètres
param_grid = {
    'pca__n_components': [0.85, 0.90, 0.95, 128, 256], # pourcentage de variance ou nombre de composantes
    'svc__C': [0.1, 0.5, 1, 5, 10], #régularisation
    'svc__kernel': ['linear', 'rbf'], # séparation linéaire ou non linéaire
    'svc__gamma': ['scale', 'auto'] # ?
}

scorers = {
    'f1_score': make_scorer(f1_score),
    'precision_score': make_scorer(precision_score)
}
# Configuration de la recherche d'hyperparamètres


grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), # Ajout de shuffle et random_state pour reproductibilité
    scoring=scorers,      # Maintenant un dictionnaire de métriques
    refit='f1_score',     # Indique quelle métrique doit être utilisée pour sélectionner le meilleur modèle final
    n_jobs=-1,
    return_train_score=True # Pour voir les scores d'entraînement également si désiré
)


# Entraînement du modèle avec recherche d'hyperparamètres
grid.fit(X, y)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('pca', PCA(n_components=0.95)),
                                       ('svc',
                                        SVC(class_weight='balanced',
                                            kernel='linear',
                                            probability=True))]),
             n_jobs=-1,
             param_grid={'pca__n_components': [0.85, 0.9, 0.95, 128, 256],
                         'svc__C': [0.1, 0.5, 1, 5, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['linear', 'rbf']},
             refit='f1_score', return_train_score=True,
             scoring={'f1_score': make_scorer(f1_score, response_method='predict'),
                      'precision_score': make_scorer(precision_score, response_method='predict')})

In [104]:
joblib.dump(grid.best_estimator_, 'classifier_V2_fixed_ask.pkl')
files.download('classifier_V2_fixed_ask.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
df_p = pd.DataFrame(grid.cv_results_)
df_p.to_csv("param_classifier_V2_fixed_ask.csv")
files.download('param_classifier_V2_fixed_ask.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [108]:
# 4. Test du modèle
def predict_with_confidence(text, model_path='classifier_V2_fixed_ask.pkl'):
    # Charger le modèle et générer l'embedding
    model = joblib.load(model_path)
    embedding = st_model.encode([text])

    # Récupérer les probabilités
    probas = model.predict_proba(embedding)[0]

    return {

        'classe': "hesitation" if probas[1] > 0.75 else "other",
        'certitude': f"{probas[1]*100:.2f}%",
    }
# Test avec des exemples
test_phrases = [
    "Angela asked what brand of jeans Jennifer was wearing because _ was wearing an unusual logo.",
    "The bedroom had to have the bureau replaced by the new closet , since the _ was too packed for all the new clothes.",
    "While camping in france, we had to decide whether to go south or south next, so we chose the _ because we wanted to see the aurora borealis.",
    "The man couldn't decide on a smoothie or a milkshake. He decided on a _ because it is creamier.",
    "Since _ arrived at work sooner Dennis would eat their lunch before Adam in the break room at their job.",
    "Jean really liked the profiteroles but not the peas because the _ were very sweet.",
    "Katrina had an obsession with donuts and cake but not Sarah so _ was a heavy weight.",
    "William asked having fresh eggs for breakfast every morning but Brett hated eggs. _ asked a chicken to raise for eggs.",
    "While making her breakfast, Sam sweetened her oatmeal with honey. The _ was sweet and sticky.",
    "Adam placed the fruit into the bag instead of the box, as the fruit fit well into the _ ."
]

for phrase in test_phrases:
    print(f"Phrase: {phrase}")
    print(f"Résultat: {predict_with_confidence(phrase)}\n")

Phrase: Angela asked what brand of jeans Jennifer was wearing because _ was wearing an unusual logo.
Résultat: {'classe': 'other', 'certitude': '56.19%'}

Phrase: The bedroom had to have the bureau replaced by the new closet , since the _ was too packed for all the new clothes.
Résultat: {'classe': 'other', 'certitude': '55.31%'}

Phrase: While camping in france, we had to decide whether to go south or south next, so we chose the _ because we wanted to see the aurora borealis.
Résultat: {'classe': 'other', 'certitude': '56.34%'}

Phrase: The man couldn't decide on a smoothie or a milkshake. He decided on a _ because it is creamier.
Résultat: {'classe': 'other', 'certitude': '55.59%'}

Phrase: Since _ arrived at work sooner Dennis would eat their lunch before Adam in the break room at their job.
Résultat: {'classe': 'other', 'certitude': '56.04%'}

Phrase: Jean really liked the profiteroles but not the peas because the _ were very sweet.
Résultat: {'classe': 'other', 'certitude': '56.25

In [15]:
probas

NameError: name 'probas' is not defined

In [18]:
df_pred_comp = pd.read_json('train_xl.jsonl',
                      lines=True)



In [72]:
import joblib
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch

classe = 'hesitation'

# 1. Charger le modèle Sentence Transformer et s'assurer qu'il utilise le GPU
# Spécifiez 'cuda' comme device si un GPU est disponible.
# Si vous avez plusieurs GPUs, vous pouvez spécifier un index comme 'cuda:0', 'cuda:1', etc.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Le modèle Sentence Transformer utilisera : {device}")


# 2. Charger votre classifieur pré-entraîné (celui-ci restera sur CPU)
model = joblib.load('classifier_V2_hesitation_no_aug.pkl')

# Votre DataFrame avec les phrases à prédire
# Exemple:
# data = {'sentence': ["Ceci est un plat délicieux", "J'aime regarder des films", "Le dessert était succulent", "Le climat est doux"]}
# df_pred_comp = pd.DataFrame(data)

# --- Processus optimisé pour le GPU ---

# 3. Encoder toutes les phrases en une seule fois (ou par lots si le DataFrame est immense)
# Cela enverra toutes les phrases au GPU pour l'encodage.
print("Encodage des phrases avec le modèle Sentence Transformer (sur GPU si disponible)...")
embeddings = st_model.encode(df_pred_comp['sentence'].tolist(), convert_to_tensor=True, show_progress_bar=True)
# Si vous utilisez convert_to_tensor=True, les embeddings seront déjà des tenseurs PyTorch sur le bon device.
# Si votre classifieur scikit-learn attend un tableau NumPy, vous devrez le reconvertir:
# embeddings_np = embeddings.cpu().numpy() # Convertit en NumPy et le déplace du GPU vers le CPU si nécessaire

print("Encodage terminé. Début de la prédiction du classifieur...")

# 4. Effectuer la prédiction avec le classifieur
# Le classifieur (probablement scikit-learn) travaillera sur le CPU
proba = model.predict_proba(embeddings.cpu().numpy())[:, 1] # Assurez-vous que les embeddings sont sur CPU et en NumPy

print("Prédiction du classifieur terminée. Détermination des classes...")

# 5. Appliquer la logique de détermination de classe
# Vectorisation de la logique pour éviter un .apply() Python-lourd sur les probabilités
def determine_class(p):
    if p >= 0.75:
        return classe
    elif p >= 0.25:
        return "incertitude"
    else:
        return "autre"

# Appliquer la fonction vectorisée aux probabilités
df_pred_comp['probabilite_nourriture'] = proba
df_pred_comp['classe'] = df_pred_comp['probabilite_nourriture'].apply(determine_class)

print("Processus terminé. Votre DataFrame est mis à jour.")
print(df_pred_comp.head())

Le modèle Sentence Transformer utilisera : cuda
Encodage des phrases avec le modèle Sentence Transformer (sur GPU si disponible)...


Batches:   0%|          | 0/1263 [00:00<?, ?it/s]

Encodage terminé. Début de la prédiction du classifieur...
Prédiction du classifieur terminée. Détermination des classes...
Processus terminé. Votre DataFrame est mis à jour.
                                qID  \
0  3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2   
1  3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1   
2  3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1   
3  3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2   
4  3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2   

                                            sentence option1 option2  answer  \
0  Ian volunteered to eat Dennis's menudo after a...     Ian  Dennis       2   
1  Ian volunteered to eat Dennis's menudo after a...     Ian  Dennis       1   
2  He never comes to my home, but I always go to ...    home   house       1   
3  He never comes to my home, but I always go to ...    home   house       2   
4  Kyle doesn't wear leg warmers to bed, while Lo...    Kyle   Logan       2   

   probabilite_nourriture classe  
0                0.045159  autre  
1                0.052039  autre  
2   

In [ ]:
df_pred_comp

,qID,sentence,option1,option2,answer,probabilite_nourriture,classe
0,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,2,0.054461,autre
1,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,1,0.042823,autre
2,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1,"He never comes to my home, but I always go to ...",home,house,1,0.069884,autre
3,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2,"He never comes to my home, but I always go to ...",home,house,2,0.091231,autre
4,3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2,"Kyle doesn't wear leg warmers to bed, while Lo...",Kyle,Logan,2,0.007734,autre
...,...,...,...,...,...,...,...
40393,3PKJ68EHDNUOUBAJ3ASD64MQ8GSJHJ-1,I felt lucky because when the fish slipped of ...,pole,net,1,0.858046,failure
40394,3W1K7D6QSDVJX2B852X30LVRM6WZBL-1,Dennis took months to write back to Kenneth's ...,Dennis,Kenneth,1,0.734849,incertitude
40395,3W1K7D6QSDVJX2B852X30LVRM6WZBL-2,Dennis took months to write back to Kenneth's ...,Dennis,Kenneth,2,0.845434,failure
40396,3BO3NEOQM2VSJ2H6ZK9L5F8V75BAIT-1,My hair looked better in a braid than a ponyta...,braid,ponytail,1,0.065517,autre


In [73]:
df_pred_comp.to_csv("df_pred_hesitation.csv")
files.download('df_pred_hesitation.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>